In [ ]:
import pandas as pd
import os
import stata_setup

# Set the Stata path
stata_setup.config("C:/Program Files/StataNow19", "se")
# Check STATA version
print("Stata initialized in mode:", stata_setup.__version__)

## Load data to pandas

In [ ]:
file_path = 'C:/Users/adamw/source/repos/microeconomics/data/DE_oczyszczone_2013_personal_final.xlsx'
try:
    df = pd.read_excel(file_path)
    print(f"Data successfully loaded from {file_path}")
except FileNotFoundError:
    print(f"File not found: {file_path}")



In [ ]:
display(df)
display(df.dtypes)

## Convert relevant columns to numeric (handling potential issues)

In [ ]:
numeric_columns = ['PW030', 'PY010G', 'PY080G', 'PY090G', 'PY100G', 'PH020', 'PB150', 'PB190']
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

# Data celaning

In [ ]:
# List of selected variables
selected_columns = [
    # Demographic
    'PB040', 'PB120', 'PB140', 'PB150', 'PB190', 'PE010',
    
    # Employment
    'PL031', 'PL015', 'PL020', 'PL025', 'PL040', 'PL060', 
    'PL100', 'PL111', 'PL130', 'PL140', 'PL150', 'PL160',
    'PL180', 'PL190',
    
    # Health
    'PH010', 'PH020', 'PH030', 'PH040', 'PH050', 'PH060', 'PH070',
    
    # Finance
    'PY010G', 'PY020G', 'PY035G', 'PY050G', 'PY090G', 
    'PY100G', 'PY110G', 'PY120G', 'PY130G', 'PY140G',
    
    # Well-being
    'PD020', 'PD030', 'PD050', 'PD060', 'PD070',
    
    # Moods
    'PW010', 'PW040', 'PW050', 'PW100', 'PW110', 'PW120',
    'PW130', 'PW140', 'PW150', 'PW160', 'PW170', 
    'PW180', 'PW190', 'PW200', 'PW220'
]
# Select only the relevant columns
df = df[selected_columns]

# Standardize column names to lowercase
df.columns = [col.lower() for col in df.columns]

Check for missing values

In [ ]:

missing_data = df.isnull().sum()
print("\nMissing data per column:")
print(missing_data[missing_data > 0])

Drop rows with excessive missing data (threshold = 10% missing)

In [ ]:
missing_threshold = int(len(df.columns) * 0.5)
print(missing_threshold)
df = df.dropna(thresh=missing_threshold, axis=0)
df

Fill missing numerical values with the median

In [ ]:
numerical_columns = df.select_dtypes(include=['number']).columns
df[numerical_columns] = df[numerical_columns].apply(lambda x: x.fillna(x.median()))

Fill missing categorical values with the mode

In [ ]:
categorical_columns = df.select_dtypes(include=['object']).columns
df[categorical_columns] = df[categorical_columns].apply(lambda x: x.fillna(x.mode()[0]))

## Convert relevant columns to numeric

In [ ]:
numeric_conversion_columns = [
    'pb040', 'pb120', 'pb140', 'pb150', 'pb190', 
    'pl060', 'pl100', 'ph010', 'ph020', 'ph030', 
    'py010g', 'py020g', 'py035g', 'py050g', 'py090g',
    'py100g', 'py110g', 'py120g', 'py130g', 'py140g',
    'pw010', 'pw040', 'pw050', 'pw100', 'pw110', 
    'pw120', 'pw130', 'pw140', 'pw150', 'pw160', 
    'pw170', 'pw180', 'pw190', 'pw200', 'pw220'
]
df[numeric_conversion_columns] = df[numeric_conversion_columns].apply(pd.to_numeric, errors='coerce')

Display summary statistics for numeric columns

In [ ]:
print("\nSummary statistics for numerical variables:")
print(df.describe())

# Import pandas DataFrame to STATA

In [ ]:
import pystata

try:
    pystata.stata.pdataframe_to_data(df, force=True)
    display("Data successfully transferred to Stata.")
except Exception as e:
    display(f"Error transferring data to Stata: {e}")

# Create a composite variables

## 1. Working `total_hours` and `over_40_hours`

In [ ]:
# Tworzenie zmiennych w Stata
pystata.stata.run('''
    * --- TWORZENIE ZMIENNYCH KOMPOZYTOWYCH ---

    * Total Hours Worked (PL060 + PL100)
    gen total_hours = pl060 + pl100
    label var total_hours "Total Hours Worked (Main + Additional Jobs)"

    * Binary Variable - Over 40 Hours
    gen over_40_hours = (total_hours > 40)
    label define over40 0 "40 hours or less" 1 "Over 40 hours"
    label values over_40_hours over40
    label var over_40_hours "Worked Over 40 Hours"
''')

print("\nZmiennie `total_hours` oraz `over_40_hours` utworzone pomyślnie.")


## Check data structure inside STATA

In [ ]:
pystata.stata.run("describe")

# Display some first rows
pystata.stata.run("list in 1/5")